# Start: loading the necessary packages

In [9]:
# make sure matplotlib shows images inline
%matplotlib inline

# import packages
import numpy as np
import time
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import os
import cv2
import random
import pickle
from PIL import Image
import xgboost as xgb

# Keras packages
from keras import layers
from keras import optimizers
from keras.layers import Input,Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_uniform

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K

In [10]:
current_directory = os.getcwd()

os.chdir(current_directory)

print(current_directory)

C:\GitHub\HandSign_Recognition


# Define some functions we'll use later in the script
* read images 
* resize images

ATTENTION: all images should have the same aspect ratio and the same orientation (either all landscape OR all portrait)

In [ ]:
# define a function that reads images from a location and uses the image names as labels
def read_images(imgloc = ''):
    # read all images from file into a numpy array
    # cv2 assumes colors are BGR, so we also convert this to RGB
    images = np.array([cv2.cvtColor(cv2.imread(imgloc + name), cv2.COLOR_BGR2RGB) 
                       for name in os.listdir(imgloc)], dtype = np.object)

    # use the image names to create a numpy array with the label of each image
    labels  = np.array([str(name.rpartition(' ')[0].rpartition('_')[2]) for name in os.listdir(imgloc)])
    
    return images, labels

In [11]:
def read_train_test(imgloc_train, imgloc_test = '', h5 = False
                    , name_train_x = '', name_train_y = '', name_test_x = '', name_test_y = ''):

    # check if the images are already split into train and test
    # and if images are saved in a folder or in a h5 file
    if imgloc_test == '':
        if h5:
            # read h5 file
            # Read the handsign dataset
            train_dataset = h5py.File(imgloc_train + '.h5', "r")
            train_img = np.array(train_dataset[name_train_x][:]) 
            train_label = np.array(train_dataset[name_train_y][:]) 
        else: # read images from folder
            # images are all in the same folder
            data_img, data_label = read_images(imgloc_train)
        
        # create a train and a testset
        nr_records_train = round(.7 * len(data_img))
        in_train = np.random.randint(len(data_img) - 1, size = nr_records_train)
        
        train_img = data_img[in_train]
        train_label = data_label[in_train]
        
        test_img = np.delete(data_img, in_train, axis = 0)
        test_label = np.delete(data_label, in_train, axis = 0)

In [ ]:
# resize images
def resize_images(img, basewidth = 64, set_grayscale = False, binarize = False, binarize_min = 135, binarize_max = 255):
    # make sure the numpy array contains integers (otherwise we can't convert them to PIL images)
    img = img.astype('uint8')
    #plt.imshow(img)
    
    # convert the numpy array image to PIL image
    img = Image.fromarray(img)
    #print(type(img))

    # calculate the height, based on the preferred width
    hsize = int((float(img.size[1]) * float((basewidth / float(img.size[0])))))
    #print(hsize)

    # resize the image
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    #print(img.size)
    #plt.imshow(img)
    
    # convert image to grayscale if parameter is True
    if set_grayscale:
        img = img.convert("L")
           
    # convert the image to numpy array
    img = np.array(img)
    #print(type(img))
    
    # binarize image if parameter is True
    if binarize:
        thr, img = cv2.threshold(img, binarize_min, binarize_max, cv2.THRESH_BINARY)
    
    # if image is converted to grayscale, make sure to set the channels shape parameter
    if set_grayscale:
        img = img.reshape((img.shape[0], img.shape[1], 1))
    
    return img

# Read images
Load all the images we want to use to train and evaluate the model

In [ ]:
# print the current datetime
print('start: ', str(time.ctime()), '\n')
start = time.time()

# read the training data
imgloc = current_directory + '/00 Data/TRAIN/'
train_img, train_label = read_images(imgloc)

# print the current datetime
print('finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

In [ ]:
# print the current datetime
print('start: ', str(time.ctime()), '\n')
start = time.time()

# read the test data
imgloc = current_directory + '/00 Data/TEST/'
test_img, test_label = read_images(imgloc)

# print the current datetime
print('finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

In [ ]:
print(train_img.shape)
print(train_label.shape)
print(train_label)

print('\n')

print(test_img.shape)
print(test_label.shape)
print(test_label)

In [ ]:
# print the current datetime
print('start: ', str(time.ctime()), '\n')
start = time.time()

# Read the handsign dataset
train_dataset = h5py.File('00 Data/train_signs.h5', "r")
train_img_h5 = np.array(train_dataset["train_set_x"][:]) # your train set features
train_label_h5 = np.array(train_dataset["train_set_y"][:]) # your train set labels

test_dataset = h5py.File('00 Data/test_signs.h5', "r")
test_img_h5 = np.array(test_dataset["test_set_x"][:]) # your test set features
test_label_h5 = np.array(test_dataset["test_set_y"][:]) # your test set labels

train_label_h5 = train_label_h5.astype('str')
test_label_h5 = test_label_h5.astype('str')

# change label 0-5 in A-F
train_label_h5[train_label_h5 == '0'] = 'A'
train_label_h5[train_label_h5 == '1'] = 'B'
train_label_h5[train_label_h5 == '2'] = 'C'
train_label_h5[train_label_h5 == '3'] = 'D'
train_label_h5[train_label_h5 == '4'] = 'E'
train_label_h5[train_label_h5 == '5'] = 'F'

test_label_h5[test_label_h5 == '0'] = 'A'
test_label_h5[test_label_h5 == '1'] = 'B'
test_label_h5[test_label_h5 == '2'] = 'C'
test_label_h5[test_label_h5 == '3'] = 'D'
test_label_h5[test_label_h5 == '4'] = 'E'
test_label_h5[test_label_h5 == '5'] = 'F'

# print the current datetime
print('finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

del train_dataset
del test_dataset

print(train_img_h5.shape)
print(train_label_h5.shape)
print(train_label_h5)

print('\n')

print(test_img_h5.shape)
print(test_label_h5.shape)
print(test_label_h5)

# print the current datetime
print('start: ', str(time.ctime()), '\n')
start = time.time()

# read the data which is copied from GitHub
imgloc = current_directory + '/00 Data/dataset/00_first_test/'
img_git, label_git = read_images(imgloc)

# print the current datetime
print('finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

print(img_git.shape)
print(label_git.shape)
print(label_git)

# DECIDE WHICH DATA TO USE FOR THE MODELING
## the default is our own data

In [ ]:
model_extension = 'own'

# USE IMAGES FROM KAGGLE
#model_extension = 'h5'
#train_img = train_img_h5
#train_label = train_label_h5
#test_img = test_img_h5
#test_label = test_label_h5

# USE IMAGES FROM GIT
#model_extension = 'git'
#train_img = train_img_git
#train_label = train_label_git
#test_img = test_img_git
#test_label = test_label_git

del train_img_h5
del train_label_h5
del test_img_h5
del test_label_h5

del train_img_git
del train_label_git
del test_img_git
del test_label_git

In [ ]:
# let's take a look at some of the images (train)

# randomly pick 4 of the images
samp = random.sample(range(0,len(train_img)-1),4)

plt.subplots(2,2)
plt.subplots_adjust(top = 0.92, bottom = 0.08, left = 0.10, right = 0.95, hspace = 0.45, wspace = 0.45)

plt.subplot(2,2,1)
plt.title('train_img[' + str(samp[0]) + '] label : ' + str(train_label[samp[0]]))
plt.imshow(train_img[samp[0]].astype('uint8'))

plt.subplot(2,2,2)
plt.title('train_img[' + str(samp[1]) + '] label : ' + str(train_label[samp[1]]))
plt.imshow(train_img[samp[1]].astype('uint8'))

plt.subplot(2,2,3)
plt.title('train_img[' + str(samp[2]) + '] label : ' + str(train_label[samp[2]]))
plt.imshow(train_img[samp[2]].astype('uint8'))

plt.subplot(2,2,4)
plt.title('train_img[' + str(samp[3]) + '] label : ' + str(train_label[samp[3]]))
plt.imshow(train_img[samp[3]].astype('uint8'))

In [ ]:
# let's take a look at some of the images (test)

# randomly pick 2 of the images
samp = random.sample(range(0,len(test_img)-1),2)

plt.subplots(2,1)
plt.subplots_adjust(top = 0.92, bottom = 0.08, left = 0.10, right = 0.95, hspace = 0.45, wspace = 0.45)

plt.subplot(2,1,1)
plt.title('test_img[' + str(samp[0]) + '] label : ' + str(test_label[samp[0]]))
plt.imshow(test_img[samp[0]].astype('uint8'))

plt.subplot(2,1,2)
plt.title('test_img[' + str(samp[1]) + '] label : ' + str(test_label[samp[1]]))
plt.imshow(test_img[samp[1]].astype('uint8'))

# Resize images
It doesn't matter which size the images are at the start, we'll try to scale them down here for the sake of efficiency

In [ ]:
# set the parameter that decides if images should be converted to grayscale
set_img_grayscale = True

# set the parameter that decides if images should be binarized (black/white)
# and set the binarize boundaries
binarize = False
binarize_min = 135
binarize_max = 255

In [ ]:
# print the current datetime
print('start: ', str(time.ctime()), '\n')
start = time.time()

# resize the images
basewidth = 64
img_list = []

# loop through all the images in the data and resize them
for img in train_img:
    # resize the numpy array images
    img = resize_images(img, basewidth, set_img_grayscale, binarize, binarize_min, binarize_max)
    # add the image to a list of numpy array images
    img_list.append(img)

#print(img_list)
    
# print the current datetime
print('finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

In [ ]:
# convert the list with images back to a numpy array
train_img = np.array(img_list)
print(train_img.shape)

In [ ]:
# let's take a look at some of the images (train)

# randomly pick 4 of the images
samp = random.sample(range(0,len(train_img)-1),4)

plt.subplots(2,2)
plt.subplots_adjust(top = 0.92, bottom = 0.08, left = 0.10, right = 0.95, hspace = 0.45, wspace = 0.45)

plt.subplot(2,2,1)
plt.title('train_img[' + str(samp[0]) + '] label : ' + str(train_label[samp[0]]))
if set_img_grayscale:
    plt.imshow(train_img[samp[0]].squeeze(), cmap = 'gray')
else:
    plt.imshow(train_img[samp[0]])

plt.subplot(2,2,2)
plt.title('train_img[' + str(samp[1]) + '] label : ' + str(train_label[samp[1]]))
if set_img_grayscale:
    plt.imshow(train_img[samp[1]].squeeze(), cmap = 'gray')
else:
    plt.imshow(train_img[samp[1]])

plt.subplot(2,2,3)
plt.title('train_img[' + str(samp[2]) + '] label : ' + str(train_label[samp[2]]))
if set_img_grayscale:
    plt.imshow(train_img[samp[2]].squeeze(), cmap = 'gray')
else:
    plt.imshow(train_img[samp[2]])

plt.subplot(2,2,4)
plt.title('train_img[' + str(samp[3]) + '] label : ' + str(train_label[samp[3]]))
if set_img_grayscale:
    plt.imshow(train_img[samp[3]].squeeze(), cmap = 'gray')
else:
    plt.imshow(train_img[samp[3]])

In [ ]:
# print the current datetime
print('start: ', str(time.ctime()), '\n')
start = time.time()

# resize the images
basewidth = 64
img_list = []

# loop through all the images in the data and resize them
for img in test_img:
    # resize the numpy array images
    img = resize_images(img, basewidth, set_img_grayscale)
    # add the image to a list of numpy array images
    img_list.append(img)

#print(img_list)
    
# print the current datetime
print('finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

In [ ]:
# convert the list with images back to a numpy array
test_img = np.array(img_list)
print(test_img.shape)

In [ ]:
# let's take a look at some of the images (test)

# randomly pick 2 of the images
samp = random.sample(range(0,len(test_img)-1),2)

plt.subplots(2,1)
plt.subplots_adjust(top = 0.92, bottom = 0.08, left = 0.10, right = 0.95, hspace = 0.45, wspace = 0.45)

plt.subplot(2,1,1)
plt.title('test_img[' + str(samp[0]) + '] label : ' + str(test_label[samp[0]]))
if set_img_grayscale:
    plt.imshow(test_img[samp[0]].squeeze(), cmap = 'gray')
else:
    plt.imshow(test_img[samp[0]])

plt.subplot(2,1,2)
plt.title('test_img[' + str(samp[1]) + '] label : ' + str(test_label[samp[1]]))
if set_img_grayscale:
    plt.imshow(test_img[samp[1]].squeeze(), cmap = 'gray')
else:
    plt.imshow(test_img[samp[1]])

In [ ]:
# normalize the data

#print(train_img[1])
train_img = train_img / 255
#print(train_img[1])

#print(test_img[1])
test_img = test_img / 255
#print(test_img[1])

# Label matrix: one hot encoding
The model needs a matrix where:
* the number of records = the number of images
* the number of columns = the number of possible values (labels) the images can represent

in this matrix all values will be 0 except for the column where the label is the label that specific image represents, there the value will be 1

In [ ]:
# determine the number of unique labels
nr_possible_values = 6 #np.unique(train_label).size #40
#print(nr_possible_values)

# create a matrice with only zeros
# the number of rows = the number of images
# the number of columns = the number of possible values we want to recognize
label_matrix_train = np.zeros([train_label.shape[0], nr_possible_values])
label_matrix_test = np.zeros([test_label.shape[0], nr_possible_values])
#print(label_matrix_train)
#print(label_matrix_test)

# create a dictionary for the labels
# we're going to use this dictionary to determine which column in the matrix corresponds to which value
#label_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9
#              , 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19
#              , 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'SPATIE': 26, 'PUNT': 27, 'STREEPJE': 28, '@': 29
#              , '1': 30, '2': 31, '3': 32, '4': 33, '5': 34, '6': 35, '7': 36, '8': 37, '9': 38, '0': 39}

#label_dict = {'0':0, '1':1, '2':2, '3':3, '4':4, '5':5}
#label_dict = {'B':0, 'C':1, 'D':2}
label_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}


# save the dictionary to file
pickle.dump(label_dict, open(current_directory + '/01 Models/LabelDictionary.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

# set the value of 1 for each record in the column with the corresponding value
count = 0
for i in train_label:
    label_matrix_train[count, label_dict[str(i)]] = 1
    count = count + 1

# set the value of 1 for each record in the column with the corresponding value
count = 0
for i in test_label:
    label_matrix_test[count, label_dict[str(i)]] = 1
    count = count + 1

#print(train_label)
#print(label_matrix_train)
print(test_label)
print(label_matrix_test)

# Prepare the model
* define variables
* define funtions

In [ ]:
# make sure to tell keras the channels are the last dimension in the shape of the dataset
# full color RGB image = 3 channels
# grayscale image = 1 channel
K.set_image_data_format('channels_last')


def plain_layer(X,n_c):
    X_in = X
    X = Conv2D(n_c,kernel_size = (3,3), padding = 'same')(X_in)
    X = BatchNormalization(axis = 3, momentum = .01)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size = (2,2))(X)
    return X


def identity_block(X,F):
    X_in = X
    
    F1,F2,F3 = F
    
    X = Conv2D(F1,kernel_size = (3,3), padding= 'same')(X_in)
    X = BatchNormalization(axis = 3, momentum = .01)(X)
    X = Activation('relu')(X)
    
    #X = Conv2D(F2,kernel_size = (3,3), padding = 'same')(X)
    #X = BatchNormalization(axis = 3, momentum = .01)(X)
    #X = Activation('relu')(X)
    
    X = Conv2D(F3,kernel_size = (3,3), padding = 'same')(X)
    X = BatchNormalization(axis = 3, momentum = .01)(X)
    
    X_in = Conv2D(F3,kernel_size = (3,3), padding = 'same')(X_in)
    X_in = BatchNormalization(axis = 3, momentum = .01)(X_in)
    
    X = Add()([X,X_in])
    X = Activation('relu')(X)
    
    return X


def Resnet(input_shape = (85,64,3), classes = 6):
    X_in = Input(input_shape)
    
    X = plain_layer(X_in,32)
    
    F1 = [16,16,32]
    X = identity_block(X,F1)
    X = MaxPooling2D(pool_size = (2,2))(X)
    
    #F2 = [16,16,32]
    #X = identity_block(X,F2)
    #X = MaxPooling2D(pool_size = (2,2))(X)
    
    #F3 = [16,16,32]
    #X = identity_block(X,F3)
    #X = MaxPooling2D(pool_size = (2,2))(X)
    
    X = plain_layer(X,32)
    X = AveragePooling2D((2,2))(X)
    
    X = Flatten()(X)
    X = Dense(512,activation='relu')(X)
    X = Dense(128,activation='relu')(X)
    X = Dense(classes,activation='softmax')(X)
    
    model = Model(inputs = X_in, outputs = X, name = 'Resnet')
    return model

In [ ]:
# create a function which makes an array from a matrix
def im_to_wide(grey):
    y = []
    for r in range(0,len(grey)):
        for col in range(0,len(grey[r])):
            y.append(grey[r][col])
    return y
            
def wide_to_im(x, size):
    if size[0] * size[1] != len(x):
        print('Size does not match')
        raise

    x = np.array(x)
    x.shape = size

    im = Image.fromarray(x)

    return im

# Train the model

In [ ]:
# https://machinelearningmastery.com/image-augmentation-deep-learning-keras/

print("start: define data generator")
# define data preparation
datagen = image.ImageDataGenerator(rescale = None #1/255 # normalize the data
                                   , rotation_range = 20 # degree range for random rotations
                                   , width_shift_range = 0.2 # range for random horizontal shifts
                                   , height_shift_range = 0.2 # range for random vertical shifts
                                   , shear_range = 0.2 # shear angle in counter-clockwise direction as radians
                                   , zoom_range = 0.2 # Range for random zoom
                                   , horizontal_flip = False # flip horizontally
                                   , vertical_flip = False # flip vertically
                                   , fill_mode = "nearest"
                                  )

print("start: fit parameters from data")
# fit parameters from data
datagen.fit(train_img)    

In [ ]:
# determine the shape of the images
img_shape = train_img.shape[1:4]

# declare a resnet model
model_resnet = Resnet(input_shape = img_shape, classes = nr_possible_values)
model_resnet_datagen = Resnet(input_shape = img_shape, classes = nr_possible_values)


# define the training parameters: nr of epochs
nr_ep = 20

# define the training parameters: batch size
btch_sz = min(int(len(train_img) / 2), 32)

# define the training parameters: steps per epoch
stps = int(len(train_img) / btch_sz)

# define the training parameters: optimizer function
opt = 'adam'

# define the training parameters: loss function
loss = 'categorical_crossentropy'


# set the parameters for the model
model_resnet.compile(optimizer = opt, loss = loss, metrics = ['accuracy'])
model_resnet_datagen.compile(optimizer = opt, loss = loss, metrics = ['accuracy'])

In [ ]:
# print the current date and time
start = time.time()
print('start: ', str(time.ctime()), '\n')

# print some of the input parameters
print("the batch_size is " + str(btch_sz) + ", the number of steps_per_epoch is " + 
      str(stps) + " and the number of epochs is " + str(nr_ep) + '\n')

# train the model
model_resnet.fit(x = train_img, y = label_matrix_train, epochs = nr_ep, batch_size = btch_sz)

# print the current date and time
print('\n', 'finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

time.sleep(5)

In [ ]:
# print the current date and time
start = time.time()
print('start: ', str(time.ctime()), '\n')

# print some of the input parameters
print("the batch_size is " + str(btch_sz) + ", the number of steps_per_epoch is " + 
      str(stps) + " and the number of epochs is " + str(nr_ep) + '\n')

# train the model with the datagenerator images
model_resnet_datagen.fit_generator(datagen.flow(train_img, label_matrix_train, batch_size = btch_sz
                                   # , save_to_dir='images', save_prefix='aug', save_format='png'
                                   ), steps_per_epoch = stps, epochs = nr_ep)

# print the current date and time
print('\n', 'finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

time.sleep(5)

In [ ]:
# create an array from the image matrix
train_img_arr  = np.array([im_to_wide(im) for im in train_img])
test_img_arr = np.array([im_to_wide(im) for im in test_img])

# remove the last dimension of the array, to make sure it's a 2D array
train_img_arr = train_img_arr[:,:,0]
test_img_arr = test_img_arr[:,:,0]

In [ ]:
# print the current date and time
start = time.time()
print('start: ', str(time.ctime()), '\n')

xb = xgb.XGBClassifier(n_estimators = 500, objective = 'multi:softprob')
xgb_model = xb.fit(train_img_arr, train_label)

print(xgb_model)

# print the current date and time
print('\n', 'finish: ', str(time.ctime()), '\n')
print("this took --- %s seconds ---" % round(time.time() - start, 2))

In [ ]:
modprob = xgb_model.predict_proba(test_img_arr)
#print(modprob.shape)
#print(modprob)

In [ ]:
#xb.classes_

# Evaluate the model

In [ ]:
# evaluate the model on the training data
eval_model_resnet = model_resnet.evaluate(train_img, label_matrix_train, batch_size = btch_sz)
eval_model_resnet_datagen = model_resnet_datagen.evaluate(train_img, label_matrix_train, batch_size = btch_sz)

print("The accuracy of the resnet model is: " + str(round(eval_model_resnet[1]*100, 2)) + "%")
print("The accuracy of the resnet datagenerator model is: " + str(round(eval_model_resnet_datagen[1]*100, 2)) + "%")

In [ ]:
# evaluate the model on the training data
#eval_my_model = my_model.evaluate(train_img, label_matrix_train, batch_size = None, steps = 100)

#print("The accuracy of this model is: " + str(round(eval_my_model[1]*100, 2)) + "%")

In [ ]:
def translate_prediction(pred, thresh = .4):
    thresh = thresh
    resulting_labels = []

    for i in range(0, pred.shape[0]):

        single_pred = pred[i]
        #print('\n the prediction matrix:')
        #print(single_pred)

        # set all elements below the threshold to zero
        single_pred[single_pred < thresh] = 0
        #print('\n the cleaned prediction matrix:')
        #print(single_pred)

        # if matrix contains all zeros, no prediction can be done
        if np.any(single_pred):
            #print("\n prediction can be done")

            # check which column contains the highest probability
            resulting_labels.append(list(label_dict.keys())[list(label_dict.values()).index(np.argmax(single_pred))])

        else:
            #print("\n no prediction possible")
            resulting_labels.append("Unknown")

        #print('\n')
    
    return resulting_labels

In [ ]:
pred_datagen_resnet = model_resnet.predict_generator(datagen.flow(test_img, label_matrix_test, batch_size = len(test_img)
                                   # , save_to_dir='images', save_prefix='aug', save_format='png'
                                   ))

pred_datagen_resnet_datagen = model_resnet_datagen.predict_generator(datagen.flow(test_img, label_matrix_test, batch_size = len(test_img)
                                   # , save_to_dir='images', save_prefix='aug', save_format='png'
                                   ))

#pred_resnet = model_resnet.predict(test_img, label_matrix_test)

#pred_resnet_datagen = model_resnet_datagen.predict(test_img, label_matrix_test)


#resnet_result = translate_prediction(pred_resnet)
#resnet_datagen_result = translate_prediction(pred_resnet_datagen)
pred_datagen_resnet_result = translate_prediction(pred_datagen_resnet)
pred_datagen_resnet_datagen_result = translate_prediction(pred_datagen_resnet_datagen)

# create a classification report
#print(classification_report(test_label, resnet_result))
print('\n')
#print(classification_report(test_label, resnet_datagen_result))
print('\n')
print(classification_report(test_label, pred_datagen_resnet_result))
print('\n')
print(classification_report(test_label, pred_datagen_resnet_datagen_result))

In [ ]:
thresh = .4
resulting_labels = []

for i in range(0, modprob.shape[0]):
   
    single_pred = modprob[i]
    print('\n the prediction matrix:')
    print(single_pred)
    
    # set all elements below the threshold to zero
    single_pred[single_pred < thresh] = 0
    print('\n the cleaned prediction matrix:')
    print(single_pred)
    
    # if matrix contains all zeros, no prediction can be done
    if np.any(single_pred):
        print("\n prediction can be done")
        
        # check which column contains the highest probability
        resulting_labels.append(list(label_dict.keys())[list(label_dict.values()).index(np.argmax(single_pred))])
        
    else:
        print("\n no prediction possible")
        resulting_labels.append("Unknown")
    
    #print('\n')


print('\n The labels according to the prediction:')
print(resulting_labels)
print('\n The labels according to the testset:')
print(test_label)

# create a classification report
print(classification_report(test_label.astype(str), resulting_labels))

# Save the model to disk
This way we can use the model on other computers

In [ ]:
# location to save the model
imgloc = current_directory + '/01 Models/'

# serialize model to JSON and save it
model_resnet_json = model_resnet.to_json()
with open(imgloc + "model_resnet_" + model_extension + ".json", "w") as json_file:
    json_file.write(model_resnet_json)

# serialize weights to HDF5
model_resnet.save_weights(imgloc + "model_resnet_" + model_extension + ".h5")
print("Saved model to disk")



# serialize model to JSON and save it
model_resnet_datagen_json = model_resnet_datagen.to_json()
with open(imgloc + "model_resnet_datagen_" + model_extension + ".json", "w") as json_file:
    json_file.write(model_resnet_datagen_json)

# serialize weights to HDF5
model_resnet_datagen.save_weights(imgloc + "model_resnet_datagen_" + model_extension + ".h5")
print("Saved model to disk")


# save xgb model
pickle.dump(xgb_model, open(imgloc + "xgb_model_" + model_extension + ".pickle.dat", "wb"))
print("Saved model to disk")

In [ ]:
# https://sandipanweb.wordpress.com/2018/01/20/hand-gesture-classification-using-deep-convolution-and-residual-neural-network-with-tensorflow-keras-in-python/